In [1]:
!pip install pyss3 pandas numpy matplotlib

     |████████████████████████████████| 2.0MB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.cElementTree as et
import shutil
import os
from pyss3 import SS3
from pyss3.util import Dataset, Evaluation, span
from pyss3.server import Live_Test
from sklearn.metrics import accuracy_score
from pathlib import Path
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import hamming_loss
# ### From Dávid Szabó to mount data
from google.colab import drive
from sys import path
from sklearn.model_selection import train_test_split

In [4]:
drive.mount('/content/drive', force_remount=True)
path_root = '/content/drive/My Drive/NLP Lab'
data_root = '/data/SB1_Train-entity'
path.append(path_root)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
cd '/content/drive/My Drive/NLP Lab/data'

/content/drive/.shortcut-targets-by-id/1WxWH0T7CKPlERIv-tNCJIqpH2T8CkX5J/NLP Lab/data


In [0]:
# (A lot of improvement can be done on this but it was just created too see how the main task works. so not much focus here on code beauty)

def parse(mode, Documents):
    Test_dic = {}
    entity_path = "Data/entity/Train"
    Labels = set()
    try:
        Path(entity_path).mkdir(parents=True,exist_ok=False)
    except FileExistsError:
        shutil.rmtree(entity_path)    
        Path(entity_path).mkdir(parents=True,exist_ok=False)
    
    for Document in Documents:
        tree = et.parse(Document)
        root = tree.getroot()
        sentences = root.findall("./Review/sentences/sentence")
        for sentence in sentences:
            content = sentence.findall("./text")[0].text
            opinions = sentence.findall("./Opinions/Opinion")
            entities = list(set([opinion.get('category').split('#')[0].lower() for opinion in opinions]))
            if len(entities):
                if mode == "Train":
                    for entity in entities:
                        Labels.add(entity)
                        file = open(entity_path+ '/' + entity + '.txt', 'a+',encoding="utf-8")
                        file.write(content + '\n')
                        file.close()
                elif mode == "Test":
                    Test_dic[content] = entities
    if mode == "Train":
      return Labels
    elif mode == "Test":
      return Test_dic

def binarizer (y_pred, Classes):
    binaryLabel = [0]*len(Classes)
    indices =  [Classes.index(label) for label in y_pred]
    for index in indices:
        binaryLabel[index] = 1
    return binaryLabel

In [0]:
mode = "Train"
if mode == 'Train':
    Documents = ['ABSA16_Laptops_Train_SB1_v2.xml','ABSA16_Restaurants_Train_SB1_v2.xml']
    Classes = list(parse(mode, Documents))
elif mode == "Test":
    Documents = ['EN_LAPT_SB1_TEST_.xml.B']
    Testcase = parse(mode, Documents)

In [21]:
clf = SS3(a=1)
x_train, y_train = Dataset.load_from_files("SB1_Train-entity", folder_label=False)
clf.train(x_train, y_train)

Training on 'WARRANTY': 100%|██████████| 28/28 [00:00<00:00, 77.53it/s]


In [31]:
loss = 0
count = 0
for key, value in Testcase.items():
    count += 1
    y_pred = clf.classify_multilabel(key)
    binary_y_pred = binarizer(y_pred,Classes)
    binary_y_true = binarizer(value,Classes)
    if count <= 3:
        print("predicted: " , y_pred)
        print("actual: ", value)
        print(hamming_loss(binary_y_true,binary_y_pred))
        print("-------------")
    loss += hamming_loss(binary_y_true,binary_y_pred)
print('...')
print('..')
print('.')
print("Final Loss: ", loss/len(Testcase))

predicted:  ['laptop']
actual:  ['laptop']
0.0
-------------
predicted:  ['laptop']
actual:  ['laptop']
0.0
-------------
predicted:  ['laptop']
actual:  ['laptop', 'company']
0.03571428571428571
-------------
...
..
.
Final Loss:  0.03097731239092498
